# Задача

По заданной выборке изображений составить бинарный классификатор, имеющий две категории: __Крокодилы__ и __Часы__

# Решения

Будем пробовать несколько моделей

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import cv2
import os
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, StratifiedShuffleSplit, train_test_split, GridSearchCV

Считаем данные

In [2]:
CROC_PATH = 'data/crocodile/'
CLOCK_PATH = 'data/clock/'
DEBUG = False
croc_dict = dict()
clock_dict = dict()
croc_data = np.array([cv2.imread(CROC_PATH + i) for i in os.listdir(CROC_PATH)])
clock_data = np.array([cv2.imread(CLOCK_PATH + i) for i in os.listdir(CLOCK_PATH)])
'''
for i in os.listdir(CROC_PATH):
    if DEBUG: print('Reading image `{}`'.format(i))
    arr = cv2.imread(CROC_PATH + i)
    croc_dict[i.replace('.png', '')] = arr
    croc_data = np.append(croc_data, arr)
for i in os.listdir(CLOCK_PATH):
    arr = cv2.imread(CLOCK_PATH + i)
    clock_dict[i.replace('.png', '')] = arr
    np.append(croc_data, arr)
'''

"\nfor i in os.listdir(CROC_PATH):\n    if DEBUG: print('Reading image `{}`'.format(i))\n    arr = cv2.imread(CROC_PATH + i)\n    croc_dict[i.replace('.png', '')] = arr\n    croc_data = np.append(croc_data, arr)\nfor i in os.listdir(CLOCK_PATH):\n    arr = cv2.imread(CLOCK_PATH + i)\n    clock_dict[i.replace('.png', '')] = arr\n    np.append(croc_data, arr)\n"

Построим простые модели и оценим их качество.

### Линейная модель (можно пропустить при выполнении кода)

In [3]:
ans = np.concatenate((np.ones(clock_data.shape[0]), np.zeros(croc_data.shape[0])))
data = np.concatenate((clock_data.reshape(clock_data.shape[0], -1), croc_data.reshape(croc_data.shape[0], -1)))
train_data, test_data, train_labels, test_labels = train_test_split(data, ans, test_size=0.3, random_state=0)
cv = StratifiedShuffleSplit(n_splits=10, random_state=0)

In [4]:
reg = LogisticRegression(random_state=0)
param_grid = {'penalty': ['l1', 'l2'], 'C': [1, 10, 100]}
grid_cv = GridSearchCV(reg, param_grid, scoring='accuracy', cv=cv, n_jobs=4)
%time grid_cv.fit(train_data, train_labels)

CPU times: user 1.76 s, sys: 96.4 ms, total: 1.86 s
Wall time: 58.3 s


GridSearchCV(cv=StratifiedShuffleSplit(n_splits=10, random_state=0, test_size='default',
            train_size=None),
       error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=4,
       param_grid={'penalty': ['l1', 'l2'], 'C': [1, 10, 100]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=0)

In [5]:
grid_cv.best_estimator_

LogisticRegression(C=100, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [6]:
grid_cv.best_score_

0.7057142857142857

Результаты не очень.

### Случайный лес (можно пропустить)

In [7]:
reg = RandomForestClassifier(random_state=0, n_jobs=4)
param_grid = {'n_estimators': [600],
              'max_depth': [30], 'criterion': ['entropy']}
grid_cv = GridSearchCV(reg, param_grid, scoring='accuracy', cv=cv, n_jobs=4)
%time grid_cv.fit(train_data, train_labels)

CPU times: user 23.3 s, sys: 63.1 ms, total: 23.3 s
Wall time: 1min 2s


GridSearchCV(cv=StratifiedShuffleSplit(n_splits=10, random_state=0, test_size='default',
            train_size=None),
       error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=4,
            oob_score=False, random_state=0, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=4,
       param_grid={'n_estimators': [600], 'max_depth': [30], 'criterion': ['entropy']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=0)

In [8]:
grid_cv.best_estimator_

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=30, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=600, n_jobs=4,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [9]:
grid_cv.best_score_

0.8242857142857143

### Градиентный бустинг над случайными лесами (тоже можно пропустить)

In [13]:
import xgboost as xgb

In [14]:
reg = xgb.XGBClassifier(random_state=0, learning_rate=0.005, n_jobs=4, n_estimators=300)
#param_grid = {'n_estimators': [100, 200, 400, 600],
#              'learning_rate': [0.001, 0.005, 0.1]}
param_grid = {'n_estimators': [400],
              'learning_rate': [0.005]}
grid_cv = GridSearchCV(reg, param_grid, scoring='accuracy', cv=cv, n_jobs=4)
%time grid_cv.fit(train_data, train_labels)

/usr/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. U

CPU times: user 1min 6s, sys: 182 ms, total: 1min 6s
Wall time: 10min 31s


GridSearchCV(cv=StratifiedShuffleSplit(n_splits=10, random_state=0, test_size='default',
            train_size=None),
       error_score='raise',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.005, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=300,
       n_jobs=4, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1),
       fit_params=None, iid=True, n_jobs=4,
       param_grid={'n_estimators': [400], 'learning_rate': [0.005]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=0)

In [15]:
reg.fit(train_data, train_labels)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.005, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=300,
       n_jobs=4, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [16]:
cross_val_score(reg, test_data, test_labels)

/usr/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


array([0.76, 0.77, 0.81])

## А теперь изюминка нашего проекта: CNN

In [17]:
from keras.models import Model
from keras.layers import Input, Convolution2D, MaxPooling2D, Dense, Dropout, Flatten
from keras.utils import np_utils

/usr/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [18]:
ans = np.concatenate((np.ones(clock_data.shape[0]), np.zeros(croc_data.shape[0])))
data = np.concatenate((clock_data, croc_data))
train_data, test_data, train_labels, test_labels = train_test_split(data, ans, test_size=0.2, random_state=42)
#cv = StratifiedShuffleSplit(n_splits=10, random_state=0)

In [19]:
train_data.shape

(800, 32, 32, 3)

In [29]:
# Параметры модели. С ними надо будет поиграться
batch_size = 21
num_epochs = 91
kernel_size = 3
pool_size = 2
conv_depth_1 = 32
conv_depth_2 = 32
drop_prob_1 = 0.3
drop_prob_2 = 0.4
hidden_size = 128

In [30]:
num_train, depth, height, width = train_data.shape
num_test = test_data.shape[0]
num_classes = np.unique(train_labels).shape[0]

In [31]:
train_data = train_data.astype('float32')
test_data = test_data.astype('float32')
train_data /= np.max(train_data)
test_data /= np.max(test_data)

In [32]:
Y_train = np_utils.to_categorical(train_labels, num_classes)
Y_test = np_utils.to_categorical(test_labels, num_classes)

In [33]:
inp = Input(shape=(depth, height, width))

# Conv [32] -> Conv [32] -> Pool (with dropout on the pooling layer)
conv_1 = Convolution2D(conv_depth_1, (kernel_size, kernel_size), padding='same', activation='relu')(inp)
#conv_2 = Convolution2D(conv_depth_1, (kernel_size, kernel_size), padding='same', activation='relu')(conv_1)
pool_1 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_1)
drop_1 = Dropout(drop_prob_1)(pool_1)

# Conv [64] -> Conv [64] -> Pool (with dropout on the pooling layer)
conv_3 = Convolution2D(conv_depth_2, (kernel_size, kernel_size), padding='same', activation='relu')(drop_1)
#conv_4 = Convolution2D(conv_depth_2, (kernel_size, kernel_size), padding='same', activation='relu')(conv_3)
pool_2 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_3)
drop_2 = Dropout(drop_prob_2)(pool_2)

# Now flatten to 1D, apply FC -> ReLU (with dropout) -> softmax
flat = Flatten()(drop_2)
hidden = Dense(hidden_size, activation='relu')(flat)
drop_3 = Dropout(drop_prob_2)(hidden)
out = Dense(num_classes, activation='softmax')(drop_3)

In [34]:
model = Model(input=inp, output=out)

/usr/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  """Entry point for launching an IPython kernel.


In [35]:
model.compile(loss='categorical_crossentropy', optimizer='adagrad', metrics=['accuracy'])

In [36]:
model.fit(train_data, Y_train, batch_size=batch_size, epochs=num_epochs, verbose=1, validation_split=0.1)

Train on 720 samples, validate on 80 samples
Epoch 1/91
720/720 [==============================] - 1s 2ms/step - loss: 0.7510 - acc: 0.5292 - val_loss: 0.6731 - val_acc: 0.6625
Epoch 2/91
720/720 [==============================] - 1s 2ms/step - loss: 0.6333 - acc: 0.6375 - val_loss: 0.5341 - val_acc: 0.7250
Epoch 3/91
720/720 [==============================] - 1s 2ms/step - loss: 0.6268 - acc: 0.6778 - val_loss: 0.6133 - val_acc: 0.6625
Epoch 4/91
720/720 [==============================] - 1s 2ms/step - loss: 0.5353 - acc: 0.7472 - val_loss: 0.4864 - val_acc: 0.7500
Epoch 5/91
720/720 [==============================] - 1s 2ms/step - loss: 0.5037 - acc: 0.7653 - val_loss: 0.5604 - val_acc: 0.6750
Epoch 6/91
720/720 [==============================] - 1s 2ms/step - loss: 0.4702 - acc: 0.7875 - val_loss: 0.5194 - val_acc: 0.6875
Epoch 7/91
720/720 [==============================] - 2s 3ms/step - loss: 0.4273 - acc: 0.8181 - val_loss: 0.3366 - val_acc: 0.8625
Epoch 8/91
720/720 [===========

720/720 [==============================] - 2s 2ms/step - loss: 0.0662 - acc: 0.9833 - val_loss: 0.2102 - val_acc: 0.9125
Epoch 63/91
720/720 [==============================] - 1s 2ms/step - loss: 0.0769 - acc: 0.9708 - val_loss: 0.1821 - val_acc: 0.9375
Epoch 64/91
720/720 [==============================] - 2s 3ms/step - loss: 0.0776 - acc: 0.9736 - val_loss: 0.2790 - val_acc: 0.8875
Epoch 65/91
720/720 [==============================] - 2s 2ms/step - loss: 0.0707 - acc: 0.9764 - val_loss: 0.2358 - val_acc: 0.8875
Epoch 66/91
720/720 [==============================] - 1s 2ms/step - loss: 0.0621 - acc: 0.9778 - val_loss: 0.1835 - val_acc: 0.9375
Epoch 67/91
720/720 [==============================] - 2s 2ms/step - loss: 0.0631 - acc: 0.9694 - val_loss: 0.2263 - val_acc: 0.9000
Epoch 68/91
720/720 [==============================] - 1s 2ms/step - loss: 0.0639 - acc: 0.9722 - val_loss: 0.1616 - val_acc: 0.9625
Epoch 69/91
720/720 [==============================] - 1s 2ms/step - loss: 0.0612

In [37]:
model.evaluate(test_data, Y_test, verbose=1)

200/200 [==============================] - 0s 931us/step


[0.40107868909835814, 0.89]